In [147]:
#If you need to install in modules in jupyter notebook 
%pip install scikit-learn
%pip install pandas
%pip install matplotlib 
%pip install seaborn 
%pip install graphviz
%pip install scikit-plot   
%pip install statsmodels   


#Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import datetime # Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# Common imports
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
import scikitplot as skplt
from sklearn.neighbors import KNeighborsClassifier  
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# to make this notebook's output stable across runs
np.random.seed(42)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):

    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [64]:
#changes the output for the print statements
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',0)

In [65]:
#Pull 
df = pd.read_csv('https://raw.githubusercontent.com/Rodrig79/Machine-Learning-Data-Analysis-Project/master/rawData/pokemon.csv') 

In [79]:
from sklearn import preprocessing

#Data Cleaning
df = df[~df.Name.str.contains('Mega')] #removed pokemon with "Mega" in it
df = df.drop(columns = ["Name","Type 1","Type 2","#","Generation"]) #Removed columns with names

#normalization
X = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df=pd.DataFrame(x_scaled, columns=df.columns)


AttributeError: 'DataFrame' object has no attribute 'Name'

In [80]:
print(df)

        Total        HP    Attack   Defense   Sp. Atk   Sp. Def     Speed  \
0    0.233898  0.173228  0.251429  0.195556  0.323529  0.214286  0.228571   
1    0.381356  0.232283  0.325714  0.257778  0.411765  0.285714  0.314286   
2    0.584746  0.311024  0.440000  0.346667  0.529412  0.380952  0.428571   
3    0.218644  0.149606  0.268571  0.168889  0.294118  0.142857  0.342857   
4    0.381356  0.224409  0.337143  0.235556  0.411765  0.214286  0.428571   
..        ...       ...       ...       ...       ...       ...       ...   
746  0.711864  0.421260  0.542857  0.515556  0.417647  0.357143  0.514286   
747  0.711864  0.192913  0.542857  0.644444  0.529412  0.619048  0.257143   
748  0.711864  0.311024  0.600000  0.244444  0.823529  0.523810  0.371429   
749  0.847458  0.311024  0.885714  0.244444  0.941176  0.523810  0.428571   
750  0.711864  0.311024  0.600000  0.511111  0.705882  0.333333  0.371429   

     Legendary  
0    0.0        
1    0.0        
2    0.0        
3    0.

In [89]:
 #Splitting dataset to x and y
y = df.Attack
X = df[['Defense','HP','Sp. Atk','Sp. Def','Speed']]

In [90]:
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=0)
linreg = LinearRegression().fit(X_train, y_train)

In [144]:
def MultiVariateLinearRegression(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60)
  regr = linear_model.LinearRegression()
  regr.fit(X, y)
  print('Intercept: \n', regr.intercept_)
  print('Coefficients: \n', regr.coef_)
  kf = KFold(n_splits=10, random_state=None, shuffle=True)
  kf.get_n_splits(X)
  for train_index, test_index in kf.split(X):
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_train)
  print("coefficient of determination score",r2_score(y_train,y_pred))
  print("mean squared error",mean_squared_error(y_train,y_pred,squared =False))


In [145]:
def MultiVariateLinearRegressionWithGradient(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60)
    regr = )
    regr.fit(X, y)
#     scaler = StandardScaler()
#     scaler.fit(X_train)
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
    kf.get_n_splits(X)
    for train_index, test_index in kf.split(X):
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_train)
        print("coefficient of determination score",r2_score(y_train,y_pred))
        print("mean squared error",mean_squared_error(y_train,y_pred,squared =False))


SyntaxError: unexpected EOF while parsing (<ipython-input-145-d6abd40692d5>, line 2)

In [146]:
def RegularizedLinearRegression(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60)
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
    kf.get_n_splits(X)
    for train_index, test_index in kf.split(X):
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_train)
        print("coefficient of determination score",r2_score(y_train,y_pred))
        print("mean squared error",mean_squared_error(y_train,y_pred,squared =False))


SyntaxError: unexpected EOF while parsing (<ipython-input-146-11735042b536>, line 2)

In [ ]:
def PolynomialRegression(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60)
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
    kf.get_n_splits(X)
    for train_index, test_index in kf.split(X):
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_train)
        print("coefficient of determination score",r2_score(y_train,y_pred))
        print("mean squared error",mean_squared_error(y_train,y_pred,squared =False))


In [142]:
MultiVariateLinearRegression(X,y)

Intercept: 
 0.01593310558287342
Coefficients: 
 [ 0.57361907  0.53694675  0.17710785 -0.37966666  0.29630742]
coefficient of determination score 0.4212479997186188
mean squared error 0.12957667984155083

